In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from TSForecasting.mage_calc import MageDataSet

In [35]:
data = pd.read_csv('~/Desktop/NCSA_genomics/Python - notebooks//TSForecasting/Data/consolidatedDataForPackage.csv')

meta = pd.read_csv('~/Desktop/NCSA_genomics/Python - notebooks//TSForecasting/Data/consolidatedMetadata.csv')

In [36]:
data['Display Time'] = data['Display Time'].apply(lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
print("Here is a glimpse of the data:\n")
print(data.head())

Here is a glimpse of the data:

         Display Time subjectId  GlucoseValue
0 2016-06-21 00:00:00     GVP01         197.0
1 2016-06-21 00:15:00     GVP01         208.0
2 2016-06-21 00:30:00     GVP01         226.0
3 2016-06-21 00:45:00     GVP01         241.0
4 2016-06-21 01:00:00     GVP01         250.0


In [37]:
print("Subject\tMage Score")
for subjectId, df in data.groupby('subjectId'):
    glucs = df['GlucoseValue'].to_list()
    indices = [1*i for i in range(len(glucs))]
    m = MageDataSet(indices, glucs)
    k = m.getMAGE()
#     k=0
    print(subjectId+"\t"+str(k))
    


Subject	Mage Score
1636-69-001	94.91666666666667
1636-69-001-2	66.65714285714286
1636-69-026	55.710144927536234
1636-69-028	36.70175438596491
1636-69-032	29.62264150943396
1636-69-035	62.82539682539682
1636-69-048	27.65714285714286
1636-69-053	42.56716417910448
1636-69-060	35.48717948717949
1636-69-064	56.810810810810814
1636-69-069	49.666666666666664
1636-69-090	57.23529411764706
1636-69-091	36.44
1636-69-100	28.29850746268657
1636-69-104	36.09756097560975
1636-69-104-2	35.80597014925373
1636-69-107	42.875
1636-69-111	26.976470588235294
1636-69-114	35.77333333333333
1636-69-123	29.847619047619048
1636-70-1002	42.45977011494253
1636-70-1003	39.75581395348837
1636-70-1005	60.50704225352113
1636-70-1008	61.707317073170735
1636-70-1010	48.625
2133-001	36.361702127659576
2133-002	33.59375
2133-003	58.19298245614035
2133-004	78.86206896551724
2133-006	33.4375
2133-007	47.83098591549296
2133-008	30.567010309278352
2133-009	45.1
2133-010	30.287037037037038
2133-011	45.38961038961039
2133-012	